In [1]:
#!/usr/bin/env python2
import os
import sys
import argparse
import time
from getpass import getuser
from distutils.dir_util import copy_tree

import numpy as np
import random
import theano
import theano.tensor as T
from theano import config
import lasagne
from lasagne.regularization import regularize_network_params
from lasagne.objectives import binary_crossentropy

from iterator import Iterator
import models_v2 as models



import PIL.Image as Image
from matplotlib import pyplot as plt
from matplotlib import colors
%matplotlib inline

In [2]:
#Training hyper-parameters

learning_rate = 0.001
weight_decay = 0#TODO :NOT implemented yet!!
num_epochs = 5
max_patience = 100
#data_augmentation={}
savepath = 'save_models' #might need to change a bit
loadpath = 'load_models'
batch_size = 128
extract_center = False
load_caption = False
nb_discriminator_steps = 2

#Model Hyperparameters
conv_before_pool = [1,1,1,1]
n_filters = 64
filter_size = 3
n_units_dense_layer = 10
out_nonlin = lasagne.nonlinearities.sigmoid

In [3]:
#######################
# Saving path and stuff
######################
exp_name = 'gan'

savepath=os.path.join(sys.path[0],savepath, exp_name)
loadpath=os.path.join(sys.path[0],loadpath, exp_name)

if not os.path.exists(savepath):
    os.makedirs(savepath)
else:
    print('\033[93m The following folder already exists {}. '
          'It will be overwritten in a few seconds...\033[0m'.format(
              savepath))
print 'Saving directory : ' + savepath
print 'Loading directory : '+ loadpath

 The following folder already exists save_models\gan. It will be overwritten in a few seconds...
Saving directory : save_models\gan
Loading directory : load_models\gan


In [4]:
#######################
#Build dataset iterator
#######################

print "Loading training data..." #threads???
train_iter = Iterator(which_set='train', batch_size = batch_size,
            extract_center = extract_center, load_caption = load_caption)
print "Loading validation data..." #threads???
valid_iter = Iterator(which_set='valid', batch_size = batch_size,
            extract_center = extract_center, load_caption = load_caption)
test_iter = None

n_batches_train = train_iter.n_batches
n_batches_valid = valid_iter.n_batches
n_batches_test = test_iter.n_batches if test_iter is not None else 0

print "Batch. train: %d, val %d, test %d" % (n_batches_train,
                            n_batches_valid, n_batches_test)

Loading training data...
Loading validation data...
Batch. train: 646, val 316, test 0


In [5]:
##########################################
# Build generator and discriminator models
##########################################
print('Defining symbolic variables and building models')

#Input and target var for the generator
G_input_var = T.tensor4('generator_input_var') #m noise samples
D_input_var = T.tensor4('true images')



Defining symbolic variables and building models


In [6]:
##########################################
# Build generator and discriminator models
##########################################

print('Building gan model')
gan = models.gan(G_input_var = G_input_var, D_input_var = D_input_var)

#Print layers and shape (to debug)
# print 'Generator layers'
# for layer in lasagne.layers.get_all_layers(generator):
#     print layer, layer.output_shape
# print 'Discriminator layers'
# for layer in lasagne.layers.get_all_layers(discriminator):
#     print layer, layer.output_shape

Building gan model


In [ ]:
#####################################
# Define and compile theano functions
#####################################
print "Defining and compiling theano functions"

#Prediction functions
#G_predictions = lasagne.layers.get_output(gan.G['last_layer'])
D_predictions = lasagne.layers.get_output(gan.D['last_layer'])
D_over_G_predictions = lasagne.layers.get_output(gan.D_over_G['last_layer'])

#Loss functions
D_loss = -(T.log(D_predictions) + T.log(1- D_over_G_predictions)).mean()
G_loss = (T.log(1-D_over_G_predictions)).mean() #binary_crossentropy(D_predictions, D_target_var)
#Ainsi, on fera de la descente de gradient sur chacune de ces loos

#TODO : weight decay
# if weight_decay > 0:
#     gen_weightsl2 = regularize_network_params(
#         generator, lasagne.regularization.l2)
#     discr_weightsl2 = regularize_network_params(
#         discriminator, lasagne.regularization.l2)
#     gen_loss += weight_decay * gen_weightsl2
#     discr_loss += weight_decay * discr_weightsl2


#Update parameters for each model

D_params = lasagne.layers.get_all_params(gan.D['last_layer'], trainable=True)
D_updates = lasagne.updates.adam(D_loss, D_params, learning_rate = learning_rate)
#discr_acc = $dicriminator accuracy
D_train_fn = theano.function([G_input_var,D_input_var], D_loss, updates = D_updates)



G_params = lasagne.layers.get_all_params(gan.G['last_layer'], trainable=True)
G_updates = lasagne.updates.adam(G_loss, G_params, learning_rate = learning_rate)
#gen_acc = #generator accuracy
G_train_fn = theano.function([G_input_var], G_loss, updates=G_updates)

print 'Done'
#If ImportError: DLL load failed: The specified module could not be found
#Open with ipython notebook instead of jupyter notebook (that may fix the problem)


Defining and compiling theano functions


In [ ]:
print 'yay'

In [25]:
print img_train_batch[0][0][7][32]

0.909804


In [ ]:
num_epochs = 5
nb_discriminator_steps = 1

for epoch in range(num_epochs):
    #Train discriminator
    print 'Train discriminaTOR'
    for k in range(nb_discriminator_steps):
        
        for i,batch in enumerate(train_iter):
            if i!=0:
                break
            img_train_batch, center_train_batch, caps_train_batch = batch
            img_train_batch, center_train_batch, caps_train_batch = img_train_batch[:10], center_train_batch[:10], caps_train_batch[:10]
            img_train_batch = np.transpose(img_train_batch, (0,3,1,2))
            center_train_batch = np.transpose(center_train_batch, (0,3,1,2))
            #print img_train_batch.shape, center_train_batch.shape, np.array(list(caps_train_batch)).shape
            
            G_input_noise = np.random.rand(img_train_batch.shape[0],n_filters*8,4,4)
            #print 'noise input var done'
            D_cost_train_batch = D_train_fn(G_input_noise,center_train_batch)
            print 'epoch = ', epoch , 'k = ', k, 'i = ', i, 'pixel' , img_train_batch[0][0][7][32], 'cost = ', D_cost_train_batch
         
    #Train generator    
    print 'Train generator'
    for i,batch in enumerate(train_iter):
        if i!=0:
            break
            
        G_input_noise = np.random.rand(128,n_filters*8,4,4)
        #print 'noise input var done'
        G_cost_train_batch = G_train_fn(G_input_noise)
        print 'epoch = ', epoch , 'k = ', k, 'i = ', i, 'pixel' , img_train_batch[0][0][7][32], 'cost = ', G_cost_train_batch
            
print 'done'
            
            

In [12]:
batch[2].shape

AttributeError: 'tuple' object has no attribute 'shape'

In [29]:
img_train_batch.shape

(126L, 64L, 64L, 3L)